In [3]:
import numpy as np
from scipy import stats

In [4]:
S = 105
X = 100  # 選擇權履約價格
t = 0.5  # 選擇權到期時間（年）
r = 0.03  # 無風險利率
b = 0.02  # 常數（持有成本 b=r-q）
Sigma = 0.2  # 波動率
_ITERATION_MAX_ERROR = 0.00001

In [5]:
def BS_call(S, X, t, r, b, Sigma):
    d1 = (np.log(S/X) + (b + 0.5 * Sigma**2)*t)/(Sigma * np.sqrt(t))
    d2 = d1 - Sigma * np.sqrt(t)
    Call = S * np.exp((b - r) * t)*stats.norm.cdf(d1,0.0,1.0) - X * np.exp(-r*t) * stats.norm.cdf(d2,0.0,1.0)
    return Call

## Newton‘s Method

In [6]:
def _Kc(X, t, r, b, Sigma):

    N = 2 * b / Sigma**2
    M = 2 * r / Sigma**2
    q2u = (-1 * (N - 1) + ((N - 1)**2 + 4 * M)**0.5) / 2  # p27.
    su = X / (1 - 1 / q2u)                                # p27.
    h2 = -1 * (b * t + 2 * Sigma * np.sqrt(t)) * X / (su - X)  # p31.
    Si = X + (su - X) * (1 - np.exp(h2))                       # p31.

    K = (1 - np.exp(-1 * r * t))
    d1 = (np.log(Si / X) + (b + Sigma**2 / 2) * t) / (Sigma * np.sqrt(t))
    q2 = (-1 * (N - 1) + ((N - 1)**2 + 4 * M / K)**0.5) / 2  # p16.
    LHS = Si - X
    RHS = BS_call(Si, X, t, r, b, Sigma) + (1 - np.exp((b - r) * t) * stats.norm.cdf(d1,0.0,1.0)) * Si / q2  # p23.
    bi = np.exp((b - r) * t) * stats.norm.cdf(d1,0.0,1.0) * (1 - 1 / q2) + (1 - np.exp((b - r) * t) * stats.norm.pdf(d1,0.0,1.0) / (Sigma * np.sqrt(t))) / q2  # p25. 

    E = _ITERATION_MAX_ERROR
    
    while np.abs(LHS - RHS) / X > E:        # p26.
        Si = (X + RHS - bi * Si) / (1 - bi) # p25.
        d1 = (np.log(Si / X) + (b + Sigma**2 / 2) * t) / (Sigma * np.sqrt(t))
        LHS = Si - X
        RHS = BS_call(Si, X, t, r, b, Sigma) + (1 - np.exp((b - r) * t) * stats.norm.cdf(d1,0.0,1.0)) * Si / q2
        bi = np.exp((b - r) * t) * stats.norm.cdf(d1,0.0,1.0) * (1 - 1 / q2) + (1 - np.exp((b - r) * t) * stats.norm.cdf(d1,0.0,1.0) / (Sigma * np.sqrt(t))) / q2
    
    return Si

In [7]:
S_star = _Kc(X, t, r, b, Sigma)

print("臨界股價 S* =", S_star)

臨界股價 S* = 331.31439799020546


## 使用臨界股價求出美式選擇權近似解

In [8]:
def _approximateAmericanCall(S, X, t, r, b, Sigma):
    '''
    Barone-Adesi And Whaley
    '''

    if b >= r:                               # 無股利發放
        return BS_call(S, X, t, r, b, Sigma)
    else:                                    # 有股利發放
        Sk = _Kc(X, t, r, b, Sigma)
        N = 2 * b / Sigma**2
        M = 2 * r / Sigma**2
        K = (1 - np.exp(-1 * r * t))
        d1 = (np.log(Sk / X) + (b + (Sigma**2) / 2) * t) / (Sigma * (t**0.5))
        q2 = (-1 * (N - 1) + ((N - 1)**2 + 4 * M / K)**0.5) / 2
        a2 = (Sk / q2) * (1 - np.exp((b - r) * t) * stats.norm.cdf(d1,0.0,1.0))  # P21.
        if S < Sk:
            return BS_call(S, X, t, r, b, Sigma) + a2 * (S / Sk)**q2  # 小於臨界股價 P21.
        else:
            return S - X                  

In [9]:
Amcall_price = _approximateAmericanCall(S, X, t, r, b, Sigma)
print("美式買權價格 =", Amcall_price)

美式買權價格 = 9.190348228253553
